<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第五章：在无标签数据上预训练

In [1]:
from importlib.metadata import version

pkgs = [
    "matplotlib",
    "numpy",
    "tiktoken",
    "torch",
    "tensorflow"
]
for p in pkgs:
    try:
        print(f"{p} verison: {version(p)}")
    except ImportError:
        print(f"{p} is not installed")

matplotlib verison: 3.7.5
numpy verison: 1.26.4
tiktoken verison: 0.8.0
torch verison: 2.0.1+cu118
tensorflow is not installed


- 本章节实现训练和评估一个预训练LLM的初步代码
- 结尾也将OpenAI发布的预训练权重加载到本节实现的模型中

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/chapter-overview.webp">
</p>

- 本章节覆盖的主体如下图所示：

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/mental-model--0.webp" width=800px>
</p>

## 5.1 评估生成文本模型
- 先使用之前章节中的代码初始化一个GPT模型进行一个简短的回顾
- 然后讨论评价LLMs的基础指标/metrics
- 最后将评价指标应用到训练和测试数据集

### 5.1.1 使用GPT生成文本
- 使用之前的代码初始化一个GPT模型

In [ ]:
import torch

from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # 设置为评估模式，关闭dropout

- 上面使用了0.1的dropout，但现在训练LLM时不使用dropout已经相对常见
- 现代的LLM也不在`nn.Linear`层中使用偏置向量（不同于早期的GPT模型），这是通过设置`"qkv_bias": False`实现的
- 将上下文长度(`context_length`)减少到256个token，以减少训练模型的计算资源需求，而原始的124百万参数的GPT-2模型使用了1024个token
  - 这样做是为了让更多的读者能够跟随和执行代码示例，而原始的124百万参数的GPT-2模型使用了1024个token
  - 然而，请随意增加`context_length`到1024个token（这不需要任何代码更改）
  - 稍后也会从预训练权重加载一个1024 `context_length`的模型

- 接下来，使用上一章节的`generate_text_simple`函数生成文本
- 此外，还定义了两个便利函数，`text_to_token_ids`和`token_ids_to_text`，用于在token和文本表示之间进行转换，本章节中会使用它们

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/gpt-process.webp" width=800px>
</p>

In [4]:
import tiktoken

from previous_chapters import generate_text_simple


def text_to_token_ids(text: str, tokenizer: tiktoken.Encoding) -> torch.Tensor:
    encoded = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # 加一个batch维度
    return encoded_tensor


def token_ids_to_text(token_ids: torch.Tensor, tokenizer: tiktoken.Encoding) -> str:
    flat = token_ids.squeeze(0)  # 移除添加的batch维度
    return tokenizer.decode(flat.tolist())


start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx = text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


- 如上所见，模型尚未产生良好的文本，这是因为它还没有经过训练
- 如何衡量或捕捉什么是“好的文本”，以数值形式跟踪它，以便在训练过程中测量它？
- 下一小节介绍指标，用于计算生成输出的损失指标，我们可以使用它来衡量训练进度
- 下一章节在微调LLMs时也会介绍其他衡量模型质量的方法